In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo

In [12]:
executable_path = {'executable_path': '.\\resources\\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [13]:
## Data Scraping NASA's News Page ##

# Opens web page
nasa_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

browser.visit(nasa_url)
html = browser.html

nasa_soup = BeautifulSoup(html, 'html.parser')

In [24]:
# Latest News Headline 
latest_headline = nasa_soup.find_all('div',class_ = 'content_title')[1].text
headline_desc = nasa_soup.find_all('div', class_ = 'article_teaser_body')[0].text

print("Latest Headline:")
print(latest_headline)
print("Headline Description:")
print(headline_desc)

Latest Headline:
NASA Engineers Checking InSight's Weather Sensors
Headline Description:
An electronics issue is suspected to be preventing the sensors from sharing their data about Mars weather with the spacecraft.
